<a href="https://colab.research.google.com/github/Joykareko/Data-Science-Projects/blob/main/Kenyas_Politics_Sentiment_Analysis_ML_Project_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importing the dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
from textblob import TextBlob
from wordcloud import WordCloud

In [3]:
#loading the data
election_data = pd.read_csv('/content/kenya_political_tweets.csv')
election_data.head(3)

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet
0,1396091006030856192,Doomster,Utopia slums,"Alt-left, liberal, pronoun:them",2009-08-23 12:22:53,309,907,8050,False,2021-05-22 13:10:08,@PrettyAmmina The change to the new blue unifo...,NaN,Twitter Web App,0.0,0.0,False
1,1395981137764397056,Unruly 🗯,Morayfield,I don't know what you thought this was gonna b...,2011-04-20 19:35:54,1053,523,5279,False,2021-05-22 05:53:33,I think it's now safe to say President Uhuru K...,NaN,Twitter for Android,0.0,0.0,False
2,1395433210490798082,Khavin,Nairobi,CCTV installation and MAINTENANCE technician. ...,2020-02-11 04:52:44,4202,3735,29563,False,2021-05-20 17:36:17,@_CrazyKenyan President Uhuru Kenyatta bribed ...,['bbinonsense'],Twitter Web App,0.0,1.0,False


In [4]:
#checking the rows and columns
election_data.shape

(11723, 16)

In [5]:
#checking the general info
election_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11723 entries, 0 to 11722
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                11723 non-null  object 
 1   user_name         11723 non-null  object 
 2   user_location     9239 non-null   object 
 3   user_description  10730 non-null  object 
 4   user_created      11722 non-null  object 
 5   user_followers    11722 non-null  object 
 6   user_friends      11722 non-null  object 
 7   user_favourites   11722 non-null  object 
 8   user_verified     11721 non-null  object 
 9   date              11722 non-null  object 
 10  text              11722 non-null  object 
 11  hashtags          1174 non-null   object 
 12  source            11722 non-null  object 
 13  retweets          11721 non-null  float64
 14  favorites         11721 non-null  float64
 15  is_retweet        11721 non-null  object 
dtypes: float64(2), object(14)
memory usage: 

In [6]:
#missing values?
election_data.isnull().sum()

id                      0
user_name               0
user_location        2484
user_description      993
user_created            1
user_followers          1
user_friends            1
user_favourites         1
user_verified           2
date                    1
text                    1
hashtags            10549
source                  1
retweets                2
favorites               2
is_retweet              2
dtype: int64

In [7]:
#interested in user loaction and text
election_data['user_location'].value_counts()

Nairobi, Kenya                   2954
Kenya                            1189
Nairobi                           558
Limuru, Kenya                     186
Mombasa, Kenya                    173
                                 ... 
Mogadishu, Baidoa and Nairobi       1
Weldonlangat@gmail.com              1
Oude Meer, Haarlemmermeer           1
Geertruidenberg, Nederland          1
naenda kanani                       1
Name: user_location, Length: 986, dtype: int64

In [8]:
#stopwords do not add value to the text
import nltk
nltk.download('stopwords')
print(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'bo

In [11]:
#replacing the null values in 'text' column with empty strings
#replacing the null values in 'location' column with mode
election_data.fillna(value = {'text':' ','user_location':'Nairobi'},inplace = True)
election_data.isnull().sum()

id                      0
user_name               0
user_location           0
user_description      993
user_created            1
user_followers          1
user_friends            1
user_favourites         1
user_verified           2
date                    1
text                    0
hashtags            10549
source                  1
retweets                2
favorites               2
is_retweet              2
dtype: int64

Cleaning the Data